In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, auc, roc_curve, log_loss, accuracy_score, precision_score, f1_score, recall_score
import seaborn as sns

In [ ]:
subset = 0

trainer = pd.read_csv(f"labels/train_subset{subset}.csv")
tester = pd.read_csv(f"labels/test_subset{subset}.csv")
validator = pd.read_csv(f"labels/val_subset{subset}.csv")

trainer["Filename"] = "images/" + trainer["Filename"]
tester["Filename"] = "images/" + tester["Filename"]
validator["Filename"] = "images/" + validator["Filename"]

label_map = {
    0: "Chinee_Apple",
    1: "Lantana",
    2: "Parkinsonia",
    3: "Parthenium",
    4: "Prickly_Acacia",
    5: "Rubber_Vine",
    6: "Siam_Weed",
    7: "Snake_Weed",
    8: "Negative"
}

trainer["Label"] = trainer["Label"].map(label_map)
tester["Label"] = tester["Label"].map(label_map)
validator["Label"] = validator["Label"].map(label_map)

In [ ]:
image_size = (224, 224)
batch_size = 32
datagen = ImageDataGenerator(rescale=1/255, rotation_range = 30)

train_gen = datagen.flow_from_dataframe(
    trainer,
    x_col = "Filename",
    y_col = "Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

val_gen = datagen.flow_from_dataframe(
    validator,
    x_col = "Filename",
    y_col = "Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_gen = datagen.flow_from_dataframe(
    tester,
    x_col = "Filename",
    y_col = "Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=layers.Input(shape=(224, 224, 3))
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.1),
    layers.Dense(9, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True) #stop early if the val loss doesn't decrease beyond 7 epochs and restore the model to its best weights
# if val loss increases, its a sign of overfitting - the model's memorising the train data

reducelronplateau = tf.keras.callbacks.ReduceLROnPlateau( #if the performance isn't improving, the model reduces LR
    monitor='val_loss', 
    factor=0.5, #reduce by this factor. eg 0.5x will half the LR
    patience=3, #how many epochs long the performance stagnation will last before LR forced to reduce
    verbose=1,
    min_lr=1e-5  #minimum LR- prevents too much reduction
) #done to ensure that the model reaches its local minima instead of continually overshooting it

history = model.fit(
    train_gen,
    epochs=50, #the model won't likely complete all epochs due to early stopping
    validation_data = val_gen,
    callbacks = [early_stop, reducelronplateau]
)


In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train accuracy') #plots training and val accuracy v/s epoch
plt.plot(history.history['val_accuracy'], label='val Accuracy')
plt.title('accuracy Curves')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train Loss') #plots training and val loss v/s epoch
plt.plot(history.history['val_loss'], label='val Loss')
plt.title('loss Curves')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
probabalistics = model.predict(test_gen)

print("example of the cnn output showing probabilities:")
print(probabalistics[0]) #softmax output for probability values of classes

predictions = np.argmax(probabalistics, axis=1) #converts row into single value of highest prob. index = class ID
real_labels = np.argmax(test_gen, axis=1) #same here, except only 0s and 1s

indexed_items = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

print(classification_report(real_labels, predictions)) #classification report and confusion matrix

cm = confusion_matrix(real_labels, predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap="inferno")
plt.title("Confusion matrix")
plt.xticks(np.arange(10) + 0.5, indexed_items, rotation=90)
plt.yticks(np.arange(10) + 0.5, indexed_items, rotation=0)
plt.show()

In [ ]:
np.fill_diagonal(cm, 0) #removing all correctly classified examples - in the diagonal
sns.heatmap(cm, annot=True, cmap="inferno")
plt.xticks(np.arange(10) + 0.5, indexed_items, rotation=90)
plt.yticks(np.arange(10) + 0.5, indexed_items, rotation=0)
plt.title("Misclassification report")

In [ ]:
errors = np.where(predictions != real_labels)[0] #displaying the first 5 images where the model misclassified.
for i in errors[:5]:
    plt.imshow(test_gen[i], cmap='gray')
    plt.title(f"True: {indexed_items[real_labels[i]]}, Pred: {indexed_items[predictions[i]]}")
    plt.show()
    
#the classes tshirt and shirt are very similar. the model is expected to make some errors here. similar with ankle boots and sandals, and pullovers and coats.

In [ ]:
print(f"Matthew's Correlation Coefficient : {matthews_corrcoef(real_labels, predictions)}")
print(f"Accuracy : {accuracy_score(real_labels, predictions)}")
print(f"f1 score : {f1_score(real_labels, predictions, average='weighted')}")
print(f"Log loss : {log_loss(real_labels, probabalistics)}")
print(f"Recall : {recall_score(real_labels, predictions, average='weighted')}")
print(f"Precision : {precision_score(real_labels, predictions, average='weighted')}")

In [ ]:
from sklearn.calibration import label_binarize

real_bin = label_binarize(real_labels, classes=np.arange(10))

plt.figure(figsize=(15, 15))

for i in range(10):
    
    false_p_rate, true_p_rate, thresholds = roc_curve(real_bin[:, i], probabalistics[:, i])
    
    roc_auc = auc(false_p_rate, true_p_rate) #higher area under curve - straighter the lines - lesser the FPR
    
    plt.plot(false_p_rate, true_p_rate, label=f"{indexed_items[i]} - AUC: {roc_auc:.2f}")

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curves (1 v/s rest)')
plt.legend(loc='lower right')
plt.grid()
plt.show()
